In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import open3d as o3d
import k3d

from utils.data_utils import *
from utils.visualization_utils import *
from ray_tracing import *

## Test ray_tracing.intersection_points [DEPRECATED]

In [2]:
# init the object and normalize
bun = o3d.data.BunnyMesh()
mesh_bun = o3d.io.read_triangle_mesh(bun.path)
mesh_bun.vertices = o3d.utility.Vector3dVector(normalize(np.asarray(mesh_bun.vertices)))

# init camera views
camera_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=2, resolution=10)
views = np.asarray(camera_sphere.vertices)

In [3]:
# setup open3d virtual camera
width = 320
height = 240
vis, ctr, param = setup_virtual_camera(width, height)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
# calculcate camera fov in degrees
fx = param.intrinsic.intrinsic_matrix[0, 0]
x0 = param.intrinsic.intrinsic_matrix[0, 2]
fov = np.arctan2(x0, fx) * 180 / np.pi

In [14]:
# setup raycasting scene and calculate the rays
scene = o3d.t.geometry.RaycastingScene()
center = np.zeros(3)
up = np.array([0, 1, 0])
eye = views[7]
rays = scene.create_rays_pinhole(fov, center, eye, up, width, height)
rays = rays.numpy().reshape(-1, 6)
rays = Ray(origin=Vec3D(rays[:, :3]), direction=Vec3D(rays[:, 3:]))

In [6]:
# setup the grid
num_voxels = 32
voxel_size = 2. / num_voxels
min_bound = Vec3D(np.array([-1., -1., -1.]))
max_bound = Vec3D(-min_bound.data)
grid = Grid(voxel_size, num_voxels, min_bound, max_bound)

In [7]:
# find and process t_min and t_max
t_min, t_max = intersection_points(rays, grid)
# mask out the non-intersecting rays
min_mask = np.logical_or(t_min == np.inf, t_min == -np.inf)
max_mask = np.logical_or(t_max == np.inf, t_max == -np.inf)
mask = np.logical_not(np.logical_or(min_mask, max_mask))
# clip rays within a threshold
thresh = 4  # twice the radius
t_min = np.clip(t_min, a_min=0, a_max=thresh)
t_max = np.clip(t_max, a_min=0, a_max=thresh)

In [8]:
# plot the object
vertices = flip_axes(np.asarray(mesh_bun.vertices))
indices = np.asarray(mesh_bun.triangles)
plot = k3d.plot()
plot += k3d.mesh(
            vertices, 
            indices,
            attribute=vertices[:, 2],
            color_map=k3d.colormaps.matplotlib_color_maps.inferno,
        )

# plot entrance points
points = flip_axes(rays.origin.data + t_min[:, None] * rays.direction.data)[mask]
plot += k3d.points(points, point_size=0.01, shader='mesh', color=0xff0ac5)

# plot exit points
points = flip_axes(rays.origin.data + t_max[:, None] * rays.direction.data)[mask]
plot += k3d.points(points, point_size=0.01, shader='mesh', color=0x000ac5)

# plot camera
plot += k3d.points(flip_axes(eye), point_size=0.1, shader='mesh', color=0x00ffc5)

plot.display()

/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  warnings.warn(
/Users/khasmamad/miniconda3/envs/nbv/lib/python3.8/site-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

## Test ray_tracing.shoot_rays [DEPRECATED]

In [27]:
# setup the grid
num_voxels = 32
voxel_size = 2. / num_voxels
min_bound = Vec3D(np.array([-1., -1., -1.]))
max_bound = Vec3D(-min_bound.data)
grid = Grid(voxel_size, num_voxels, min_bound, max_bound)

# setup ground truth grid
voxel_bun = o3d.geometry.VoxelGrid.create_from_triangle_mesh_within_bounds(
    mesh_bun, voxel_size, min_bound.data, max_bound.data)
occupied_indices = np.asarray([voxel.grid_index for voxel in voxel_bun.get_voxels()])
gt_grid = Grid(voxel_size, num_voxels, min_bound, max_bound)
gt_grid.data[gt_grid.data == VoxelType.unseen] = 0
gt_grid.set_labels(occupied_indices, VoxelType.occupied)

In [ ]:
# setup open3d virtual camera
width = 320
height = 240
vis, ctr, param = setup_virtual_camera(width, height)

In [104]:
# calculcate camera fov in degrees
fx = param.intrinsic.intrinsic_matrix[0, 0]
x0 = param.intrinsic.intrinsic_matrix[0, 2]
fov = np.arctan2(x0, fx) * 180 / np.pi

# setup raycasting scene and calculate the rays
scene = o3d.t.geometry.RaycastingScene()
center = np.zeros(3)
up = np.array([0, 1, 0])
eye = np.array([0, 0, -1]) * np.sqrt(5)
rays = scene.create_rays_pinhole(fov, center, eye, up, width, height)
rays = rays.numpy().reshape(-1, 6)
num_rays = len(rays)
rays = Ray(origin=Vec3D(rays[:num_rays, :3]), direction=Vec3D(rays[:num_rays, 3:]))

In [105]:
print(rays.origin.data)
print(rays.direction.data)

[[-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]
 ...
 [-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]
 [-0.       -0.       -2.236068]]
[[-0.31169513 -0.23352703  0.92103815]
 [-0.30992916 -0.23366897  0.9215979 ]
 [-0.30816004 -0.23381028  0.9221552 ]
 ...
 [ 0.30815998  0.2338103   0.9221552 ]
 [ 0.30992913  0.233669    0.9215979 ]
 [ 0.3116951   0.23352706  0.92103815]]


In [106]:
shoot_rays(rays, grid, gt_grid, 5)

start_idx:
[ 9  9  9 ... 22 22 22]
end_idx:
[ 0  0  0 ... 31 31 31]
step:
[-1 -1 -1 ...  1  1  1]
tmax:
[1.40361512 1.41161288 1.41971691 ... 1.41971712 1.41161298 1.40361532]
tdelta:
[0.20051645 0.20165898 0.2028167  ... 0.20281674 0.20165901 0.20051646]
start_idx:
[10 10 10 ... 21 21 21]
end_idx:
[ 4  3  3 ... 28 28 27]
step:
[-1 -1 -1 ...  1  1  1]
tmax:
[1.60580976 1.60483439 1.60386444 ... 1.60386427 1.60483422 1.6058096 ]
tdelta:
[0.26763496 0.26747242 0.26731074 ... 0.2673107  0.26747236 0.26763493]
start_idx:
[0 0 0 ... 0 0 0]
end_idx:
[27 27 28 ... 28 27 27]
step:
[1 1 1 ... 1 1 1]
tmax:
[1.40989601 1.40903964 1.40818815 ... 1.40818815 1.40903964 1.40989601]
tdelta:
[0.06785821 0.067817   0.06777601 ... 0.06777601 0.067817   0.06785821]
67012
x: [1.40361512 1.41161288 1.41971691 ... 1.41971712 1.41161298 1.40361532]
y: [1.60580976 1.60483439 1.60386444 ... 1.60386427 1.60483422 1.6058096 ]
z: [1.40989601 1.40903964 1.40818815 ... 1.40818815 1.40903964 1.40989601]

67012
x: [1.

In [11]:
occupied_x, occupied_y, occupied_z = np.where(grid.data == VoxelType.occupied)
empty_x, empty_y, empty_z = np.where(grid.data == VoxelType.empty)
unseen_x, unseen_y, unseen_z = np.where(grid.data == VoxelType.unseen)

occ_grid = np.zeros([num_voxels, num_voxels, num_voxels])
occ_grid[np.where(grid.data == VoxelType.occupied)] = 1
empty_grid = np.zeros([num_voxels, num_voxels, num_voxels])
empty_grid[np.where(grid.data == VoxelType.empty)] = 1
unseen_grid = np.zeros([num_voxels, num_voxels, num_voxels])
unseen_grid[np.where(grid.data == VoxelType.unseen)] = 1

In [12]:
visualize_occupancy(empty_grid, do_flip_axes=True)
visualize_occupancy(occ_grid, do_flip_axes=True)
visualize_occupancy(unseen_grid, do_flip_axes=True)

Output()

Output()

Output()

## Test data generation pipeline

### Test view movement

In [20]:
from data_generation import move

In [26]:
sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.0, resolution=10, create_uv_map=False)
sphere_points = np.array(sphere.vertices)

view = random_sphere_point(npoints=1, radius=np.sqrt(3.1))[0]
path = [view]
new_view = view
for _ in range(4):
    new_view = move("left", new_view)
    path.append(new_view)

visualize_camera_path(path)

Output()

### test data_generation.generate_data

In [29]:
from data_generation import generate_data
import json

In [30]:
bun = o3d.data.BunnyMesh()
mesh_bun = o3d.io.read_triangle_mesh(bun.path)
mesh_bun.vertices = o3d.utility.Vector3dVector(normalize(np.asarray(mesh_bun.vertices)))

In [31]:
# setup ground truth grid
num_voxels = 32
grid_data, voxel_size, min_bound, max_bound = to_occupancy_grid(
    mesh_bun, input_type="mesh", do_normalization=True, num_voxels=num_voxels
)
gt_grid = Grid(voxel_size, num_voxels, Vec3D(min_bound), Vec3D(max_bound))
gt_grid.data = grid_data

In [33]:
generate_data(mesh_bun, gt_grid, debug=True)

Received scan no 0
	next_view_dir : up
	next_view_coords : [-1.3292478486514363, 0.952810885397435, 0.6521131600601892]
	# of occed : 248
	next_view_utility : 139

Received scan no 1
	next_view_dir : left
	next_view_coords : [-0.16872546423298931, 1.6267391137474005, 0.6521131600601893]
	# of occed : 387
	next_view_utility : 90

Received scan no 2
	next_view_dir : down
	next_view_coords : [-0.16687845087389916, 1.6089314349331527, -0.6953353294043639]
	# of occed : 477
	next_view_utility : 70

Received scan no 3
	next_view_dir : left
	next_view_coords : [1.104412729422324, 1.1818550261229415, -0.6953353294043639]
	# of occed : 547
	next_view_utility : 34

Received scan no 4
	next_view_dir : left
	next_view_coords : [1.616535922580386, -0.05762283131977312, -0.6953353294043639]
	# of occed : 581
	next_view_utility : 61

Received scan no 5
	next_view_dir : left
	next_view_coords : [1.017460113963273, -1.2574910322447512, -0.6953353294043639]
	# of occed : 642
	next_view_utility : 72

Cov

In [34]:
with open('data/data/trial/bun.json', 'r') as f:
    bun_scans = json.load(f)

In [38]:
camera_path = list()
for i, scan in bun_scans.items():
    direction = scan['next_view_dir']
    view = np.array(scan['next_view_coords'])
    partial_model = np.array(scan['partial_model'])
    utility = scan['next_view_utility']
    camera_path.append(view)
    visualize_partial_model(partial_model, True)

Output()

Output()

Output()

Output()

Output()

Output()

In [39]:
visualize_camera_path(camera_path)

Output()